In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!python -m pip install --user docx2pdf pdf2image pyarrow

In [ ]:
SAVE_ONE_STACKED_JPEG_FILE=False

In [ ]:
PATH_TO_DOCX="./echantillon"

In [ ]:
# Windows Only section
from docx2pdf import convert
import os

for file in os.listdir(PATH_TO_DOCX):
    try:
        convert(file)
    except:
        print(f"bug with {file}")

In [ ]:
import pdf2image


import os

if "OS" in os.environ.keys() and "WINDOWS" in os.environ["OS"].upper():
    OS="WINDOWS"

    POPPLER_PATH = r"C:\Users\conrad.thiounn\Desktop\poppler-23.01.0\Library\bin"
else:
    OS="UNIX"

    POPPLER_PATH = r"/usr/share/poppler"

In [ ]:
#https://gist.github.com/willprice/fd917d2cafe56732137e60154d456688

from PIL import Image
import numpy as np

def vstack(images):
    if len(images) == 0:
        raise ValueError("Need 0 or more images")

    if isinstance(images[0], np.ndarray):
        images = [Image.fromarray(img) for img in images]
    width = max([img.size[0] for img in images])
    height = sum([img.size[1] for img in images])
    stacked = Image.new(images[0].mode, (width, height))

    y_pos = 0
    for img in images:
        stacked.paste(img, (0, y_pos))
        y_pos += img.size[1]
    return stacked

In [ ]:
for file in os.listdir(PATH_TO_DOCX):
    if file.endswith(".pdf"):
        try:
            if SAVE_ONE_STACKED_JPEG_FILE:
                vstack(pdf2image.convert_from_path(pdf_path=PATH_TO_DOCX+"/"+file,thread_count=os.cpu_count())).save(file.replace(".pdf",".jpg"))
            else:
                pdf2image.convert_from_path(pdf_path=PATH_TO_DOCX+"/"+file,output_folder=PATH_TO_DOCX,thread_count=os.cpu_count(),output_file=file.replace(".pdf",""),fmt="jpeg")
        except:
            print(f"bug with {file}")

In [ ]:
import docx

def read_txt_from_docx(filename: str) -> str:
    fullText = []
    try:
        doc = docx.Document(filename)
        for para in doc.paragraphs:
            fullText.append(para.text)
    except:
        print(f"bug docx {filename}")
    return "\n".join(fullText)

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["num_dossier", "txt","img","label_txt","index"],dtype=object)
DIR_DOCX="./echantillon/"
num_parquet=0
for i, num_dossier in enumerate(
    {file_name[:12] for file_name in os.listdir(DIR_DOCX) if file_name.endswith(".docx")}
):
    if i%200==0:
        df.to_parquet(f"Dares_1Kaccords_dataset_{num_parquet}.parquet")
        num_parquet+=1
        df = pd.DataFrame(columns=["num_dossier", "txt","img","label_txt","index"],dtype=object)
    label_file=[
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".txt") and file_name.startswith(num_dossier)
        ][0]
    docx_file=[
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".docx") and file_name.startswith(num_dossier)
        ][0]
    pdf_file=[
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".pdf") and file_name.startswith(num_dossier)
        ][0]
    
    new_line = [num_dossier] 
    txt=read_txt_from_docx(docx_file)
    if txt:
        new_line.append(txt)
    else:
        new_line.append("")
    img=vstack(pdf2image.convert_from_path(pdf_path=pdf_file,thread_count=os.cpu_count()))
    new_line.append(img.tobytes())

    with open(label_file,"rb") as file:
        new_line.append(file.read())
        
    try:
        new_line = {
            "num_dossier": new_line[0],
            "txt": new_line[1],
            "img": new_line[2],
            "label_txt" : new_line[3],
            "index" : [i]
        }
    except:
        print(f"bug {num_dossier}")
        
    print(i)
    df_new_row = pd.DataFrame.from_records(data=new_line)
    df = pd.concat([df, df_new_row])

df.to_parquet(f"Dares_1Kaccords_dataset_{num_parquet}.parquet")